In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder,LabelEncoder
import pickle

In [3]:
data = pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [5]:
# Encode categorical variables
label_encoder_gender= LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

In [7]:
# One hot encoding

from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geo = OneHotEncoder()
geo_encoder = onehot_encoder_geo.fit_transform(data[['Geography']])
geo_encoded_df = pd.DataFrame(geo_encoder.toarray() , columns = onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [8]:
## Combine one hot encoder columns with the original data

data = pd.concat([data.drop('Geography', axis = 1), geo_encoded_df], axis = 1)
data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


In [9]:
X = data.drop('EstimatedSalary',axis=1)
y = data['EstimatedSalary']

In [10]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

# Scale these features
scaler= StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [11]:
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)
with open('onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo, file)
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler, file)

# ANN Regression

1. sequential network
2. dense -> hidden neurons
3. activation function
4. optimizer -> useful for back propogation -> for updating the weights
5. loss function
6. metrics
7. training information -> logs -> folder -> tensorboard -> visualization

In [12]:
# ANN Regression
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

No activation function for output layer

In [ ]:
# build our ANN model
model = Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)),  # Hidden Layer1 connected to input layer
    Dense(32,activation='relu'),    ## Hidden layer 2
    Dense(1) ## Output layer
])

In [ ]:
# compile the model
model.compile(optimizer='adam', loss="mean_absolute_error", metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [17]:
# Set up tensorboard to capture the logs
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

log_dir = "regression_logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback= TensorBoard(log_dir=log_dir, histogram_freq=1)

In [18]:
## set up early stoppingv 
early_stopping_callback = EarlyStopping(
                            monitor='val_loss',         # monitor val_loss  
                            patience=10,                 # patience 5 epochs
                            restore_best_weights=True)  #find the best Weights in used epochs and restore

In [ ]:
# Train the model
history = model.fit(
    X_train,
    y_train,
    validation_data=(X_test,y_test),
    epochs=100, # complete cycle of training the network using all the training data
    callbacks=[tensorflow_callback,early_stopping_callback]
)

Epoch 1/100


250/250 [==============================] - 2s 3ms/step - loss: 100361.3281 - accuracy: 0.0000e+00 - val_loss: 98455.4531 - val_accuracy: 0.0000e+00
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 99417.2500 - accuracy: 0.0000e+00 - val_loss: 96573.7031 - val_accuracy: 0.0000e+00
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 96193.1797 - accuracy: 0.0000e+00 - val_loss: 91921.8984 - val_accuracy: 0.0000e+00
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 89999.1328 - accuracy: 0.0000e+00 - val_loss: 84342.4297 - val_accuracy: 0.0000e+00
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 81267.6172 - accuracy: 0.0000e+00 - val_loss: 75137.9766 - val_accuracy: 0.0000e+00
Epoch 6/100
250/250 [==============================] - 1s 2ms/step - loss: 71634.3359 - accuracy: 0.0000e+00 - val_loss: 66155.4531 - val_accuracy: 0.0000e+00
Epoch 7/100
250/250 [======================

In [20]:
%load_ext tensorboard

In [21]:
%tensorboard --logdir regression_logs/fit

In [22]:
## Evaluate the model

test_loss, test_mae = model.evaluate(X_test,y_test)
print(f"Test MAE: {test_mae}")

63/63 [==============================] - 0s 1ms/step - loss: 50368.2617 - accuracy: 0.0000e+00
Test MAE: 0.0


In [23]:
model.save('regression_model.h5')

f:\DataScience\Bank_Neural_network\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
